In [1]:
from datetime import timedelta, datetime, date
from typing import Dict
from dotenv import load_dotenv
import pandas as pd
import numpy as np

from mezo.clients import SubgraphClient, BigQueryClient
from mezo.queries import BridgeQueries
from mezo.currency_config import TOKEN_MAP, TOKENS_ID_MAP, TOKEN_TYPE_MAP
from mezo.test_utils import tests
from mezo.currency_utils import format_currency_columns, replace_token_labels
from mezo.datetime_utils import format_datetimes
from mezo.currency_utils import get_token_prices
from mezo.visual_utils import ProgressIndicators, ExceptionHandler, with_progress

python-dotenv could not parse statement starting at line 17
python-dotenv could not parse statement starting at line 18
python-dotenv could not parse statement starting at line 19
python-dotenv could not parse statement starting at line 20
python-dotenv could not parse statement starting at line 21
python-dotenv could not parse statement starting at line 22
python-dotenv could not parse statement starting at line 23
python-dotenv could not parse statement starting at line 24
python-dotenv could not parse statement starting at line 25
python-dotenv could not parse statement starting at line 26
python-dotenv could not parse statement starting at line 27
python-dotenv could not parse statement starting at line 17
python-dotenv could not parse statement starting at line 18
python-dotenv could not parse statement starting at line 19
python-dotenv could not parse statement starting at line 20
python-dotenv could not parse statement starting at line 21
python-dotenv could not parse statement 

In [2]:
raw_withdrawals = SubgraphClient.get_subgraph_data(
    SubgraphClient.MEZO_BRIDGE_OUT_SUBGRAPH,
    BridgeQueries.GET_NATIVE_WITHDRAWALS,
    'assetsUnlockeds'
)

🔍 Trying assetsUnlockeds query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.
✅ Found 502 assetsUnlockeds records


In [3]:
def clean_bridge_data(raw, sort_col, date_cols, currency_cols, asset_col, txn_type):
    """Clean and format bridge transaction data."""
    if not ExceptionHandler.validate_dataframe(raw, "Raw bridge data", [sort_col]):
        raise ValueError("Invalid input data for cleaning")
    
    df = raw.copy().sort_values(by=sort_col)
    df = replace_token_labels(df, TOKEN_MAP)
    df = format_datetimes(df, date_cols)
    df = format_currency_columns(df, currency_cols, asset_col)

    df['type'] = txn_type

    return df

In [4]:
def add_usd_conversions(df, token_column, tokens_id_map, amount_columns=None):
    """
    Add USD price conversions to any token data
    
    Args:
        df: DataFrame containing token data
        token_column: Name of column containing token identifiers
        tokens_id_map: Dictionary mapping tokens to CoinGecko IDs
        amount_columns: List of amount columns to convert, or None for auto-detection
    
    Returns:
        DataFrame with USD conversion columns added
    """
    if token_column not in df.columns:
        raise ValueError(f"Column '{token_column}' not found in DataFrame")
    
    # Get token prices
    prices = get_token_prices()
    if prices is None or prices.empty:
        raise ValueError("No token prices received from API")
    
    token_usd_prices = prices.T.reset_index()
    df_result = df.copy()
    df_result['index'] = df_result[token_column].map(tokens_id_map)
    
    df_with_usd = pd.merge(df_result, token_usd_prices, how='left', on='index')
    
    # Set MUSD price to 1.0 (1:1 with USD)
    df_with_usd.loc[df_with_usd[token_column] == 'MUSD', 'usd'] = 1.0
    
    # Auto-detect amount columns if not provided
    if amount_columns is None:
        amount_columns = [col for col in df.columns if 'amount' in col.lower() and col != 'amount_usd']
    
    # Add USD conversion for each amount column
    for col in amount_columns:
        if col in df_with_usd.columns:
            usd_col_name = f"{col}_usd" if not col.endswith('_usd') else col
            df_with_usd[usd_col_name] = df_with_usd[col] * df_with_usd['usd']
    
    return df_with_usd

@with_progress("Cleaning bridge data")
def clean_bridge_data(raw, sort_col, date_cols, currency_cols, asset_col, txn_type):
    """Clean and format bridge transaction data."""
    if not ExceptionHandler.validate_dataframe(raw, "Raw bridge data", [sort_col]):
        raise ValueError("Invalid input data for cleaning")
    
    df = raw.copy().sort_values(by=sort_col)
    df = replace_token_labels(df, TOKEN_MAP)
    df = format_datetimes(df, date_cols)
    df = format_currency_columns(df, currency_cols, asset_col)

    df['type'] = txn_type

    return df

@with_progress("Calculating growth rate")
def calculate_growth_rate(series: pd.Series, days: int):
    """Calculate percentage growth over specified days"""
    if len(series) < days + 1:
        return 0
    current = series.iloc[-1]
    past = series.iloc[-days-1]
    if past == 0:
        return 0
    return ((current - past) / past) * 100

@with_progress("Calculating max drawdown")
def calculate_max_drawdown(series: pd.Series):
    """Calculate maximum percentage drawdown"""
    if len(series) == 0:
        return 0
    running_max = series.expanding().max()
    drawdown = (series - running_max) / running_max * 100
    return abs(drawdown.min())

@with_progress("Calculating consecutive outflow days")
def calculate_consecutive_outflow_days(daily_df: pd.DataFrame):
    """Count consecutive days of net outflows"""
    if 'net_flow' not in daily_df.columns:
        return 0
    
    consecutive = 0
    for i in range(len(daily_df) - 1, -1, -1):
        if daily_df['net_flow'].iloc[i] < 0:
            consecutive += 1
        else:
            break
    return consecutive

In [6]:
raw_withdrawals.loc[raw_withdrawals['token'] == '0xdD468A1DDc392dcdbEf6db6e34E89AA338F9F186']

,timestamp_,amount,token,chain,sender,recipient,contractId_,transactionHash_,id


In [5]:
withdrawals = clean_bridge_data(
    raw_withdrawals, sort_col='timestamp_',
    date_cols=['timestamp_'], currency_cols=['amount'], 
    asset_col='token', txn_type='withdrawal'
)
withdrawals_with_usd = add_usd_conversions(
    withdrawals, token_column='token',
    tokens_id_map=TOKENS_ID_MAP, amount_columns=['amount']
)
bridge_map = {'0': 'ethereum', '1': 'bitcoin'}
withdrawals_with_usd['chain'] = withdrawals_with_usd['chain'].map(bridge_map)
withdrawals_clean = withdrawals_with_usd[[
    'timestamp_', 'amount', 'token', 'amount_usd', 'chain',
    'recipient', 'sender', 'transactionHash_', 'type']]
withdrawals_clean = withdrawals_clean.rename(columns={'sender': 'withdrawer', 'recipient': 'withdraw_recipient'})


🔄 Cleaning bridge data...
✅ Raw bridge data validation passed (438 rows)
✅ Cleaning bridge data successful


In [6]:
withdrawals_clean.sort_values(by='timestamp_', ascending=False)

,timestamp_,amount,token,amount_usd,chain,withdraw_recipient,withdrawer,transactionHash_,type
437,2025-10-03,5000.000000,USDC,4998.520000,ethereum,0x740fafd314b4a13aaf96672360d1a118654e478fc984...,0x6ccf0fa295e6c4278ddca2efeee7f465ba07ca0d,0xf31a3ab84d85408eea1950b46b4d26c0044c2439d0c6...,withdrawal
429,2025-10-02,0.010012,tBTC,1255.002914,ethereum,0xfc7ebfa2ab4b20c5a7e0cb8e8fab3e40d4619bc63c49...,0x9ecc509dac3d92466ad65eb4d5d559c882b5f7fa,0x77c0c22baceddb8cc5104f940a4e1ca0ebb530bf5a3a...,withdrawal
422,2025-10-02,0.151344,tBTC,18971.299412,ethereum,0xc36268369fb0e3d507bef075a1078353cf0668119a87...,0x146baade6ecc534394a26c5e22c431c1eee0e3c7,0x5880959e3a10920665f7aba24a0874f6575d408499f9...,withdrawal
423,2025-10-02,0.011399,tBTC,1428.950124,ethereum,0xf398364f88bd5c5803dbe8f913dce1719601e2bdf99c...,0x4943484c29775abb54e672503f409bcf8aef8416,0x18cf82a880650e4c6a38be004639833fc3ac497ea9f8...,withdrawal
425,2025-10-02,0.010644,tBTC,1334.204298,ethereum,0xa545a3e94138f76c0da22c536f7681c2e1e8ddd2e85b...,0xae45ef381c86667d790ca5205eaeb0b55e7fe1d3,0xdf8401dcb4d0d3f3d38805136f8df05fa3df9ccf02b2...,withdrawal
...,...,...,...,...,...,...,...,...,...
3,2025-09-15,0.010300,tBTC,1291.125600,bitcoin,0x8e88d336f8a14d8952ed2e2dec3c8a5c4c51791f8a32...,0x7338d582fa6827cbe9013ffea6010af158b5cbee,0xcad6a10697d712aee8a9ccf003b7518d4bfe8bf1d80b...,withdrawal
2,2025-09-15,0.010200,tBTC,1278.590400,bitcoin,0x06ce9f68daa9c24ea61bfe458277da0246c63f2f3070...,0x2ae80350c53bf62ac2670a6ac719cb5fa161ed64,0x17b596668a78498f7cd235521e99306ead874739f1bc...,withdrawal
1,2025-09-15,0.010100,tBTC,1266.055200,bitcoin,0x2051e05c3dadc424fe3fc75474630f959e3ecf26f661...,0x7d7457c46779637a339b158cabd3b1fedbe886fc,0xdc78e2743d7bd8a72c8ac977004e48bf97d5691851a0...,withdrawal
4,2025-09-15,0.010123,tBTC,1268.995555,bitcoin,0x8e88d336f8a14d8952ed2e2dec3c8a5c4c51791f8a32...,0x6e80164ea60673d64d5d6228beb684a1274bb017,0xb57c77487553ae9107ab176b3b8065f944f2b14b7d39...,withdrawal
